# Showcase of the new LNHR DAC II QCoDeS driver (v0.2.0)

Copyright (c) Basel Precision Instruments GmbH (2025), written for the QCoDeS driver `Baspi_Lnhrdac2.py`, v0.2.0

...........................................................................................................................................................................................................................................

This notebook shows brief examples on the changes made and on how to use the new and improved QCoDeS driver for the Basel Precision Instruments LNHRDAC II.

## 1 - Imports and setting up a Station

In [1]:
from time import sleep

from qcodes.station import Station
from Baspi_Lnhrdac2 import BaspiLnhrdac2, BaspiLnhrdac2SWGConfig

The names of certain methods and variables are different to previous versions of the driver to fall in line with the QCoDeS recommendations. The behaviour of the DAC upon device creation has also changed.

In [2]:
station = Station()
DAC = BaspiLnhrdac2('LNHRDAC', 'TCPIP0::192.168.0.5::23::SOCKET')
station.add_component(DAC)


Connected to: Basel Precision Instruments GmbH (BASPI) LNHR DAC II (SP1060) - 24 channel version (serial:SN 10600000011, firmware:Revision 3.4.9u) in 6.51s
All channels have been turned off (1 MOhm Pull-Down to AGND) upon initialization and are pre-set to 0.0 V if turned on without setting a voltage beforehand.



'LNHRDAC'

## 2 - Using the DAC as a DC Voltage source

When used as a DC Voltage source, the voltage and bandwidth of each channel are now controlled via QCoDeS parameters. Channels are also turned on and off via a parameter.

In [3]:
DAC.ch14.voltage.set(5.86)
DAC.ch14.high_bandwidth.set("off")
DAC.ch14.enable.set("on")

voltage = DAC.ch14.voltage.get()
print(f"{voltage} V")

5.86 V


With all parameters of this driver, they can be controlled via different coding styles, as generally intended within QCoDeS.

In [4]:
DAC.ch14.voltage(3.954)
DAC.ch14.high_bandwidth(False)
DAC.ch14.enable("ON")

voltage = DAC.ch14.voltage()
print(f"{voltage} V")

3.954 V


## 3 - Using the DAC as an Arbitrary Waveform Generator (AWG)

The process of outputting an arbitrary waveform has two steps. On the first step the waveform has to be generated. Below it is shown how a waveform can be generated and saved to the device memory with the integrated Standard Waveform Generator (SWG) instrument module.

In [5]:
config = BaspiLnhrdac2SWGConfig(
    shape = "sawtooth",
    frequency = 80,
    amplitude = 1.03,
    offset = 0.0,
    phase = 0.0
)

DAC.swg.configuration.set(config)
DAC.swg.apply("A")

ok


After the waveform is saved to device memory, further control happens through the AWG instrument module. Through this module, device memory can be read or set. That also allows the user to not use the SWG but save a true arbitrary waveform to the DAC's memory.

In [6]:
waveform = DAC.awga.waveform.get()
print(waveform)

['0.000000', '0.001648', '0.003296', '0.004944', '0.006592', '0.008240', '0.009888', '0.011536', '0.013184', '0.014832', '0.016480', '0.018128', '0.019776', '0.021424', '0.023072', '0.024720', '0.026368', '0.028016', '0.029664', '0.031312', '0.032960', '0.034608', '0.036256', '0.037904', '0.039552', '0.041200', '0.042848', '0.044496', '0.046144', '0.047792', '0.049440', '0.051088', '0.052736', '0.054384', '0.056032', '0.057680', '0.059328', '0.060976', '0.062624', '0.064272', '0.065920', '0.067568', '0.069216', '0.070864', '0.072512', '0.074160', '0.075808', '0.077456', '0.079104', '0.080752', '0.082400', '0.084048', '0.085696', '0.087344', '0.088992', '0.090640', '0.092288', '0.093936', '0.095584', '0.097232', '0.098880', '0.100528', '0.102176', '0.103824', '0.105472', '0.107120', '0.108768', '0.110416', '0.112064', '0.113712', '0.115360', '0.117008', '0.118656', '0.120304', '0.121952', '0.123600', '0.125248', '0.126896', '0.128544', '0.130192', '0.131840', '0.133488', '0.135136', '0.

The second step of outputting the waveform involves setting the parameters of the AWG module. The DAC has for AWG modules: AWG A, AWG B, AWG C and AWG D. In the code they are named in lower case and without spaces.

In [7]:
# Make sure channel is actually enabled/ turned on
DAC.ch8.enable.set("on")

DAC.awga.channel.set(8)
DAC.awga.cycles.set(615)
DAC.awga.trigger.set("disable")
DAC.awga.enable.set("on")